In [86]:
import sqlite3

def guardar(tabla, url, informacion):
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()
    
        # Verificar si ya existe el registro antes de la inserción
        cursor.execute(f"SELECT COUNT(*) FROM {tabla} WHERE url = ? AND informacion = ?", (url, informacion))
        registroExistente = cursor.fetchone()[0]

        if not registroExistente:
            # Evitar posibles problemas de SQL injection utilizando parámetros de la consulta
            cursor.execute(f"INSERT INTO {tabla} (url, informacion) VALUES (?, ?)", (url, informacion))
            print(f"Guardado en {tabla}: {url}, {informacion}")
            conexion.commit()
        else:
            print(f"Registro existente en {tabla}: {url}, {informacion}")

            
    conexion.close()

In [91]:
#CREAMOS UN ARCHIVO PARA GUARDAR LA URL:
def listaUrls(urlNueva):
    # Verificamos si la URL ya existe en el archivo

    try:
        # Verificamos si el archivo existe
        archivo = open("urls.txt", 'r')
        urlsExistentes = archivo.read().splitlines()
        archivo.close()
    except FileNotFoundError:
        # Si el archivo no existe, lo creamos
        archivo = open("urls.txt", 'w')
        urlsExistentes = []
        archivo.close()


    if urlNueva not in urlsExistentes:
        # Si la URL no existe, añadir al final del archivo
           
        archivo = open("urls.txt", 'a')
        archivo.write(f"{urlNueva}\n")
        print(f"URL añadida: {urlNueva}")
        archivo.close()
        return True
            
    else:
        print(f"La URL ya existe en el archivo: {urlNueva}")
        return False



In [92]:
def raspado(urls, etiquetas):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urlparse

    for url in urls: #Verificamos si hay alguna url inválida
        parsed_url = urlparse(url)
        if not (parsed_url.scheme and parsed_url.netloc):
            print(f"URL inválida: {url}")
            continue
        if listaUrls(url):
            contenido = requests.get(url)
            sopa = BeautifulSoup(contenido.text, 'html.parser')

            for etiqueta in etiquetas:
                elementos = sopa.find_all(etiqueta)
                
                for elemento in elementos:
                    if etiqueta == 'a':
                        # Si la etiqueta es 'a' (enlace), obtener la URL del atributo 'href'
                        informacion = elemento.get('href')
                        if not informacion.startswith(('https:', '//')):
                            continue
                    elif etiqueta == 'img':
                        # Si la etiqueta es 'img' (imagen), obtener la URL del atributo 'src'
                        informacion = elemento.get('src')
                        # Agregar condición para verificar si la URL comienza con 'https' o '//'
                        if not informacion.startswith(('https:', '//')):
                            continue  # Salta a la siguiente iteración si no cumple con la condición
                    else:
                        # En otros casos, obtener el texto del elemento
                        informacion = elemento.text.strip()

                    # Guardar en la base de datos
                    guardar(etiqueta, url, informacion)


In [93]:
listaUrl =  ["https://es.wikipedia.org/wiki/Pit_bull_terrier_americano","https://es.wikipedia.org/wiki/Basset_hound"]
listaEtiquetas = ["h1","p","a","img"]

In [95]:
raspado(listaUrl, listaEtiquetas)

La URL ya existe en el archivo: https://es.wikipedia.org/wiki/Pit_bull_terrier_americano
La URL ya existe en el archivo: https://es.wikipedia.org/wiki/Basset_hound


In [80]:
nueva_url = "https://es.wikipedia.org/wiki/Pit_bull_terrier_americano"
print(listaUrl(nueva_url))

URL añadida: https://es.wikipedia.org/wiki/Pit_bull_terrier_americano
https://es.wikipedia.org/wiki/Pit_bull_terrier_americano
